$\newcommand{\xbf}{{\bf x}}
\newcommand{\ybf}{{\bf y}}
\newcommand{\wbf}{{\bf w}}
\newcommand{\Ibf}{\mathbf{I}}
\newcommand{\Xbf}{\mathbf{X}}
\newcommand{\Rbb}{\mathbb{R}}
\newcommand{\vec}[1]{\left[\begin{array}{c}#1\end{array}\right]}
$

# Introduction à la librairie PyTorch -- Partie 1
Matériel de cours rédigé par Pascal Germain, 2019
************

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from math import sqrt
import aidecours

## La libraire pyTorch

https://pytorch.org/

In [ ]:
import torch

### Les tenseurs

In [ ]:
torch.tensor?

Un tenseur peut contenir un scalaire.

In [ ]:
a = torch.tensor(1.5)
a

In [ ]:
a + 2

In [ ]:
a.item()

Les tenseurs contenant des vecteurs ou des matrices se comportent similairement aux *array numpy*.

In [ ]:
v = torch.tensor([1,2,3])
v

In [ ]:
torch.sum(v)

In [ ]:
u = torch.tensor([1.,2.,3.])
u

In [ ]:
torch.log(u)

In [ ]:
u[0]

In [ ]:
u[1:]

In [ ]:
np.array(u) # Conversion d'un «tensor» pytorch en un «array» numpy.

In [ ]:
M = torch.tensor([[1.,2.,3.], [4, 5, 6]])
M

In [ ]:
M.shape

In [ ]:
2 * M + 1 # Opérations sur les élements. 

In [ ]:
M @ u # Produit matriciel

In [ ]:
torch.ones((2, 3))

In [ ]:
torch.zeros((3, 2))

In [ ]:
torch.rand((3, 4)) # Matrice de nombres aléatoires tirés uniformément dans l'intervalle [0,1]""

In [ ]:
torch.randn((3, 4)) # Matrice de nombres aléatoires tirés uniformément selon une loi normale N(0,1).

**ATTENTION:** Les *tenseurs pyTorch* sont plus capricieux quant à la gestion des variables de types différents que les *array numpy*.

In [ ]:
v = np.array([.3, .6, .9])
v.dtype

In [ ]:
w = np.array([-1, 3, 8])
w.dtype

In [ ]:
v_tensor = torch.from_numpy(v)
v_tensor.dtype

In [ ]:
w_tensor = torch.from_numpy(w)
w_tensor.dtype

In [ ]:
print('v:', v.dtype)
print('w:', w.dtype)

result = v @ w
print('v @ w:', result.dtype)
result

In [ ]:
print('v_tensor:', w_tensor.dtype)
print('w_tensor:', v_tensor.dtype)
result = v_tensor @ w_tensor
print('v_tensor @ w_tensor:', result.dtype)

In [ ]:
w_tensor = torch.tensor(w, dtype=torch.float64)
w_tensor

In [ ]:
print('v_tensor:', v_tensor.dtype)
print('w_tensor:', w_tensor.dtype)
result = v_tensor @ w_tensor
print('v_tensor @ x_tensor:', result.dtype)

### Dérivation automatique

Lors de l'initialisation d'un tenseur, l'argument `requires_grad=True` indique que nous désirons calculer le gradient des variables contenues dans le tenseur.

In [ ]:
x = torch.tensor(3., requires_grad=True)

Le graphe de calcul est alors bâti au fur et à mesure que des opérations mathématiques sont appliquées aux tenseurs.

In [ ]:
F = x ** 2

La fonction `F.backward()` parcours le graphe de calcul en sens inverse et calcule le gradient de la fonction $F$ selon chacune des variables du graphe.

In [ ]:
F.backward()

Après avoir exécuté la fonction `backward()`, l'attribut `grad` des tenseurs impliqués dans le calcul contient la valeur du gradient calculé au point courant. Ici, on aura la valeur :

$$\left[\frac{\partial F(x)}{\partial x}\right]_{x=3} = \big[\,2\,x\,\big]_{x=3} = 6$$

In [ ]:
x.grad

Illustrons le fonctionnement de la dérivation automatique par quelques autres exemples.

In [ ]:
x = torch.linspace(-1, 1, 11, requires_grad=True)
x

In [ ]:
produit_scalaire = x @ x
produit_scalaire

In [ ]:
produit_scalaire.backward()

In [ ]:
x.grad

In [ ]:
a = torch.tensor(-3., requires_grad=True)
b = torch.tensor(2., requires_grad=True)
m = a*b
m.backward()
print('a.grad =', a.grad)
print('b.grad =', b.grad)

In [ ]:
a = torch.tensor(-3., requires_grad=True)
b = torch.tensor(2., requires_grad=True)
m = 2*a + b
m.backward()
print('a.grad =', a.grad)
print('b.grad =', b.grad)

In [ ]:
a = torch.tensor(3., requires_grad=True)
b = torch.tensor(2., requires_grad=False)
m = a ** b
m.backward()
print('a.grad =', a.grad)
print('b.grad =', b.grad)

In [ ]:
a = torch.tensor(-3., requires_grad=True)
b = torch.tensor(2., requires_grad=True)
c = torch.tensor(4., requires_grad=True)
m1 = (a + b)
m2 = m1 * c
m2.backward()
print('a.grad =', a.grad)
print('b.grad =', b.grad)
print('c.grad =', c.grad)

In [ ]:
vecteur_a = torch.tensor([-1., 2, 3], requires_grad=True)
vecteur_b = torch.ones(3, requires_grad=True)
produit = vecteur_a @ vecteur_b
produit.backward()
print('vecteur_a =', vecteur_a, '; vecteur_a.grad =', vecteur_a.grad)
print('vecteur_b =', vecteur_b, '; vecteur_b.grad =', vecteur_b.grad)
print('produit =', produit.item())

In [ ]:
vecteur_a = torch.tensor([1., 4, 9], requires_grad=True)
result = torch.sum(torch.sqrt(vecteur_a))
result.backward()
print('vecteur_a =', vecteur_a, '; vecteur_a.grad =', vecteur_a.grad)
print('result =', result.item())

### Descente de gradient

Commencons par un exemple en une dimension.

$$f(x) = x^2 - x + 3$$

In [ ]:
def fonction_maison(x):
    return x**2 - x + 2

x = np.linspace(-2, 2)
plt.plot(x, fonction_maison(x) )
plt.plot(.5, fonction_maison(.5), 'r*')
plt.ylim(0, 6)

In [ ]:
eta = .4 # Pas de gradient
T =  20  # Nombre d'itérations

# Initialisation aléatoire 
x = torch.randn(1, requires_grad=True)

for t in range(T):
 
    # Calcul de la fonction objectif
    val = fonction_maison(x)
    
    # Calcul des gradients
    val.backward()
    
    #print('Iteration', t+1, ': x =', x.item(), '; f(x) =', val.item(), '; f\'(x) =', x.grad.item())
    
    print(f'Iteration {t+1:02}: x ={x.item(): .5f} ; f(x) ={val.item(): .5f} ; f\'(x) ={x.grad.item(): .5f}')
    
    # Mise à jour de la variable x
    with torch.no_grad():
        x -= eta * x.grad
    
    # Remise à zéro du gradient
    x.grad.zero_()


Reprenons l'exemple des moindres carrés présentés dans les transparents du cours.

$$\min_\wbf \left[\frac1n \sum_{i=1}^n (\wbf\cdot\xbf_i- y_i)^2\right].$$ 

In [ ]:
def moindre_carres_objectif(x, y, w): 
    return np.mean((x @ w - y) ** 2)

In [ ]:
x = np.array([(1,1), (0,-1), (2,.5)])
y = np.array([-1, 3, 2])

In [ ]:
fonction_objectif = lambda w: moindre_carres_objectif(x, y, w)
aidecours.show_2d_function(fonction_objectif, -5, 5, .5)

In [ ]:
w_opt = np.linalg.inv(x.T @ x) @ x.T @ y

aidecours.show_2d_function(fonction_objectif, -5, 5, .5, optimal=w_opt)

Nous créons une classe `moindre_carres` avec un fonctionnement semblable aux algorithmes de *scikit-learn* qui résout le problème des moindres carrés par descente de gradient, en utilisant les fonctionnalités de *pyTorch*

In [ ]:
class moindre_carres:
    def __init__(self, eta=0.4, nb_iter=50, seed=None):
        self.eta = eta
        self.nb_iter = nb_iter
        self.seed = seed
        
    def apprentissage(self, x, y):
        if self.seed is not None:
            torch.manual_seed(seed)
        
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32) 

        n, d = x.shape
        self.w = torch.randn(d, requires_grad=True)
        
        self.w_list = list() # Servira à garder une trace de la descente de gradient
        self.obj_list = list()
    
        for t in range(self.nb_iter+1):
            
            loss = torch.mean((x @ self.w - y) ** 2)
            
            self.w_list.append(np.array(self.w.detach()))
            self.obj_list.append(loss.item()) 
            if t == self.nb_iter: break 
        
            with torch.no_grad():
                loss.backward()
                self.w -= self.eta * self.w.grad
                
            self.w.grad.zero_()
                
    def prediction(self, x):
        with torch.no_grad():
            x = torch.tensor(x, dtype=torch.float32)
            pred = x @ self.w.detach()
            
        return pred.numpy()

Exécution de l'algorithme.

In [ ]:
eta = 0.4     # taille du pas
nb_iter = 20  # nombre d'itérations

algo = moindre_carres(eta, nb_iter)
algo.apprentissage(x, y)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14.5, 4))
aidecours.show_2d_trajectory(algo.w_list, fonction_objectif, w_opt=w_opt, ax=axes[0])
aidecours.show_learning_curve(algo.obj_list, ax=axes[1], obj_opt=fonction_objectif(w_opt))

## Exercice

Dans cet exercice, nous vous demandons de vous inspirer de la classe `moindre_carrees` ci-haut et de l'adapter au problème de la régression logistique présenté dans les transparents du cours.

In [ ]:
from sklearn.datasets import make_blobs
xx, yy = make_blobs(n_samples=100, centers=2, n_features=2, cluster_std=1, random_state=0)

aidecours.show_2d_dataset(xx, yy)

Illustrons la fonction à optimiser (avec $\lambda=0.01$):
    
$$
\frac1n \sum_{i=1}^n - y_i \wbf\cdot\xbf_i + \log(1+e^{\wbf\cdot\xbf_i})+ \frac\rho2\|\wbf\|^2\,.
$$ 

In [ ]:
def sigmoid(x):
    return 1 / (1+np.exp(-x))

def calc_perte_logistique(w, x, y, rho):
    pred = sigmoid(x @ w)
    pred[y==0] = 1-pred[y==0]
    return np.mean(-np.log(pred)) + rho*w @ w/2

fct_objectif = lambda w: calc_perte_logistique(w, xx, yy, 0.01)
aidecours.show_2d_function(fct_objectif, -4, 4, .05)

Compléter le code de la classe suivante. 

In [ ]:
class regression_logistique:
    def __init__(self, rho=.01, eta=0.4, nb_iter=50, seed=None):
        self.rho = rho
        self.eta = eta
        self.nb_iter = nb_iter
        self.seed = seed
        
    def apprentissage(self, x, y):
        if self.seed is not None:
            torch.manual_seed(seed)
        
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32) 

        n, d = x.shape
        self.w = torch.randn(d, requires_grad=True)
        
        self.w_list = list() # Servira à garder une trace de la descente de gradient
        self.obj_list = list()
    
        for t in range(self.nb_iter+1):
            pass # Compléter
                
    def prediction(self, x):
        with torch.no_grad():
            x = torch.tensor(x, dtype=torch.float32)
            pred = x @ self.w.detach()
            
        return np.array(pred.numpy() > .5, dtype=np.int)

Exécuter le code suivant pour vérifier le bon fonctionnement de votre algorithme. Essayer ensuite de varier les paramètres `rho`, `eta` et `nb_iter` afin d'évaluer leur impact sur le résultat obtenu.

In [ ]:
rho = 0.01
eta = 0.4     # taille du pas
nb_iter = 20  # nombre d'itérations

algo = regression_logistique(rho, eta, nb_iter)
algo.apprentissage(xx, yy)

fig, axes = plt.subplots(1, 3, figsize=(16, 4))
aidecours.show_2d_trajectory(algo.w_list, fct_objectif, -4, 4, .05, ax=axes[0])
aidecours.show_learning_curve(algo.obj_list, ax=axes[1])
aidecours.show_2d_predictions(xx, yy, algo.prediction, ax=axes[2]);

Reprenons l'exercice précédent en ajoutant l'apprentissange d'un *biais* à la régression logistique:

$$
\frac1n \sum_{i=1}^n - y_i (\wbf\cdot\xbf_i+b) + \log(1+e^{\wbf\cdot\xbf_i+b})+ \frac\rho2\|\wbf\|^2\,.
$$ 

In [ ]:
class regression_logistique_avec_biais:
    def __init__(self, rho=.01, eta=0.4, nb_iter=50, seed=None):
        self.rho = rho
        self.eta = eta
        self.nb_iter = nb_iter
        self.seed = seed
        
    def apprentissage(self, x, y):
        if self.seed is not None:
            torch.manual_seed(seed)
        
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32) 

        n, d = x.shape
        self.w = torch.randn(d, requires_grad=True)
        self.b = torch.zeros(1, requires_grad=True)
        
        self.w_list = list() # Servira à garder une trace de la descente de gradient
        self.obj_list = list()
    
        for t in range(self.nb_iter+1):
            pass # Compléter
                
    def prediction(self, x):
        with torch.no_grad():
            x = torch.tensor(x, dtype=torch.float32)
            pred = x @ self.w.detach() + self.b.item()
            
        return np.array(pred.numpy() > .5, dtype=np.int)

In [ ]:
rho = 0.01
eta = 0.4     # taille du pas
nb_iter = 20  # nombre d'itérations

algo = regression_logistique_avec_biais(rho, eta, nb_iter)
algo.apprentissage(xx, yy)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
aidecours.show_learning_curve(algo.obj_list, ax=axes[0])
aidecours.show_2d_predictions(xx, yy, algo.prediction, ax=axes[1]);